<a href="https://colab.research.google.com/github/yenlung/Deep-Learning-Basics/blob/master/colab04c%20%E6%9B%B4%E5%A4%9A%E7%94%A8%20transformers%20%E7%9A%84%E4%BE%8B%E5%AD%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 安裝 `transformers` 及準備中文轉換

In [1]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 49.7 MB/s 
     |████████████████████████████████| 636 kB 45.1 MB/s 
     |████████████████████████████████| 3.3 MB 36.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install opencc-python-reimplemented

     |████████████████████████████████| 484 kB 3.8 MB/s 
  Created wheel for opencc-python-reimplemented: filename=opencc_python_reimplemented-0.1.6-py2.py3-none-any.whl size=486151 sha256=f28c5e1214a7e14c10b0ae111edbaeeda1d18c8d2a87aca8fc05fd4ac188125e
  Stored in directory: /root/.cache/pip/wheels/4e/e2/60/d062d260be08788bb389521544a8fc173de9a9a78d6a593344
Successfully built opencc-python-reimplemented


In [4]:
from transformers import pipeline
from opencc import OpenCC

In [7]:
s2t = OpenCC('s2t').convert
t2s = OpenCC('t2s').convert

In [8]:
t2s('測試一下')

'测试一下'

### 2. Zero-Shot 情意分析!!

In [9]:
classifier = pipeline("sentiment-analysis")

然後我們真的可以直接做正評負評!

In [10]:
classifier("This device was actually even better than number two, but still had some huge problems with Asus' quality assurance.")

[{'label': 'NEGATIVE', 'score': 0.9980970621109009}]

#### 【使用指定的 `model`】

我們可以去 [HuggingFace 的 models](https://huggingface.co/models) 找找有沒有合適的 `models`。

更好的方式是, 去 [HuggingFace Pipeline 的說明](https://huggingface.co/transformers/main_classes/pipelines.html)中找到我們用的任務 (比如情意分析是屬於 `TextGenerationPipeline`, 看說明會發現類似這樣的句子:

The models that this pipeline can use are models that have been trained with an autoregressive language modeling objective, which includes the uni-directional models in the library (e.g. gpt2). See the list of available models on [huggingface.co/models](https://huggingface.co/models?filter=causal-lm).

點下最後 huggingface.co 的連結, 會看到適用的 models。這時在搜尋中打入 Chinese, 就可以找到適合中文的 models (但很有可能是簡體中文版)。


In [11]:
classifier = pipeline("sentiment-analysis", model="uer/roberta-base-finetuned-jd-binary-chinese")

In [12]:
text = t2s("這是近來我最喜歡的一本書。")
classifier(text)

[{'label': 'positive (stars 4 and 5)', 'score': 0.9790050983428955}]

In [13]:
text = t2s("老闆千萬要提醒負責結帳的店員，摸了錢要不就好好洗手，要不就別碰食物包材")
classifier(text)

[{'label': 'negative (stars 1, 2 and 3)', 'score': 0.9037978649139404}]

### 3. 更酷的 Zero-Shot Classification (只是中文看來沒有適合的 `model`)

In [14]:
classifier = pipeline("zero-shot-classification")

我們可以在沒有訓練的情況下, 要求電腦依我們說的種類, 我文字分類!

In [15]:
classifier(
    "ASUS ROG STRIX G15 (G513) has excellent specs and characteristics. Its powerful hardware copes with the latest games in FHD resolution. It’s quite comfortable to run even the latest games on the ASUS ZEPHYRUS G15 GA501I.",
    candidate_labels=["education", "tech", "sport"],
)

{'labels': ['tech', 'sport', 'education'],
 'scores': [0.9489916563034058, 0.03830539435148239, 0.012702961452305317],
 'sequence': 'ASUS ROG STRIX G15 (G513) has excellent specs and characteristics. Its powerful hardware copes with the latest games in FHD resolution. It’s quite comfortable to run even the latest games on the ASUS ZEPHYRUS G15 GA501I.'}

### 4. 一行文的文字生成器

In [16]:
generator = pipeline("text-generation")

In [18]:
generator("My name is Yen-Lung Tsai. I am an associate professor at the Department of Mathematical Sciences, National Chengchi University. Next year, I will",
          max_length=100,
          num_return_sequences=2,
          )

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My name is Yen-Lung Tsai. I am an associate professor at the Department of Mathematical Sciences, National Chengchi University. Next year, I will open a graduate program in computer science at the Chengchi university.\n\nI did graduate work on the Zentrath field study, which was developed by a group of scientists from universities such as Tsinghua University, Tianjin University, and Baiji University. My project project is called "Henceforth a new paper'},
 {'generated_text': 'My name is Yen-Lung Tsai. I am an associate professor at the Department of Mathematical Sciences, National Chengchi University. Next year, I will open a branch program for research on the application of math to the development of computing applications. I am well-known for my academic work. My PhD thesis is a book on computer architecture and development for IBM Linux on Windows. My research interests are in the context of data-driven systems and computing. I graduated in 1997 from the Univers

找個中文版的來試試。

In [19]:
generator = pipeline("text-generation", model="mymusise/gpt2-medium-chinese")

404 Client Error: Not Found for url: https://huggingface.co/mymusise/gpt2-medium-chinese/resolve/main/pytorch_model.bin


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at mymusise/gpt2-medium-chinese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [22]:
text = t2s("我是政治大學應用數學系的老師，我計畫明年")

In [23]:
egg = generator(text, max_length=100)

然後我們把生成的文字取出, 去掉空白, 再轉成繁體。

In [25]:
result = s2t(''.join(egg[0]['generated_text'].split()))

In [26]:
result

'我是政治大學應用數學系的老師，我計劃明年夏天參加“三國演習會”。1931年的中央電視工作團隊從這時起爲這位美術家在學生和當代著名人物中間選一個小夥子組織的戲劇學習，同年進行了開端，因爲他不願意對此表示關'

In [27]:
text = t2s("賴彥文目前在政大應數攻讀博士學位，他擅長攝影、咖啡、寫程式。他的女朋友是")

In [28]:
egg = generator(text, max_length=100)
result = s2t(''.join(egg[0]['generated_text'].split()))

In [29]:
result

'賴彥文目前在政大應數攻讀博士學位，他擅長攝影、咖啡、寫程式。他的女朋友是著名導演楊建昌和許智霖等人。2003年成爲香港電視廣播有限公司及九龍東臺的合約節目主持人，開始有很強子弟情感，其次更帶隊參與劇集的'

### 5. 原版的 BERT 來做中文填字

In [30]:
fill_mask = pipeline('fill-mask', model='bert-base-chinese')

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
fill_mask('生命的真諦是[MASK]。')

[{'score': 0.26580584049224854,
  'sequence': '生 命 的 真 諦 是 愛 。',
  'token': 2695,
  'token_str': '愛'},
 {'score': 0.0398266576230526,
  'sequence': '生 命 的 真 諦 是 你 。',
  'token': 872,
  'token_str': '你'},
 {'score': 0.030924703925848007,
  'sequence': '生 命 的 真 諦 是 美 。',
  'token': 5401,
  'token_str': '美'},
 {'score': 0.030239349231123924,
  'sequence': '生 命 的 真 諦 是 此 。',
  'token': 3634,
  'token_str': '此'},
 {'score': 0.027290094643831253,
  'sequence': '生 命 的 真 諦 是 ： 。',
  'token': 8038,
  'token_str': '：'}]

In [32]:
def fill(text):
    result = fill_mask(text)
    generated_text = []
    for d in result:
        s = ''.join(d['sequence'].split())
        print(s)
        generated_text.append(s)

In [33]:
fill('我天天都會喝一杯[MASK]。')

我天天都會喝一杯酒。
我天天都會喝一杯茶。
我天天都會喝一杯水。
我天天都會喝一杯的。
我天天都會喝一杯奶。


### 6. 閱讀測驗型的問答

這類的問答, 答案一定要出現在文章中才可以。

In [34]:
question_answerer = pipeline("question-answering", model='uer/roberta-base-chinese-extractive-qa')

In [35]:
context = '''報導隨後介紹本屆奧運金牌得主郭婞淳先前的善舉，除提到郭婞淳名字來自「倖存」一意，也提到郭婞淳在2016年購買救護車送給羅東聖母醫院，後再轉贈離島澎湖惠民醫院的故事。

報導取用澎湖縣長賴峰偉臉書發文的照片，當時郭婞淳奪金後，惠民醫院的醫護人員也覺得與有榮焉。文末則提到郭婞淳是台灣原住民，認為她以阿美族身分一舉奪金，同樣備受關注。'''

In [36]:
question = "郭婞淳捐贈的救護車，最後是在哪個醫院？"

我們寫個函式, 把繁體轉簡體, 送進 model 看看答案是什麼。最後回轉簡體轉繁體的答案。

In [39]:
def QA(question, context):
    c = t2s(context)
    q = t2s(question)
    result = question_answerer(question=q, context=c)
    ans = s2t(result['answer'])
    return ans

In [40]:
QA(question, context)

'離島澎湖惠民醫院'

In [41]:
question = "郭婞淳是哪一個原住民族的人？"

In [42]:
QA(question, context)

'阿美族'

#### 【天馬行空篇】

那我們是不是可以做一個電影訂票系統呢?

In [43]:
context = "你好，我想訂失控的審判兩張票，今天晚上8:20那場，都是全票。"

In [44]:
question = "要看哪場電影？"

In [45]:
QA(question, context)

'失控的審判'

目前相當好! 我們再繼續看下去...

In [46]:
question = "有多少人要看？"

In [47]:
QA(question, context)

'失控'

這也太失控了... 換個問法試試。

In [48]:
question = "要訂幾張票？"

In [49]:
QA(question, context)

'兩張'

In [50]:
question = "看電影的時間？"

In [51]:
QA(question, context)

'你好'

...

### 7. 對話機器人 (無中文版本)

In [52]:
from transformers import Conversation

In [53]:
chatbot = pipeline("conversational")

In [54]:
conv = Conversation("What is your favorite book?")

In [55]:
chatbot(conv)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Conversation id: f6deb9b9-87f5-4183-a2a5-ea226a561463 
user >> What is your favorite book? 
bot >> The Hunger Games 

In [56]:
conv.add_user_input("What is it about?")

In [57]:
chatbot(conv)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Conversation id: f6deb9b9-87f5-4183-a2a5-ea226a561463 
user >> What is your favorite book? 
bot >> The Hunger Games 
user >> What is it about? 
bot >> It's about a group of kids who are hungry and have to fight for food. 